# Data Preprocessing - Preparing for one of the top performing models

In this notebook, after an exploratory data analysis is performed on Give Me Some Credit's training set from last notebook, the next step is to do a feature importance evaluation to see whether all these risk drivers are necessary for building the model. 

Preprocessing steps are done here. These preprocessing steps will be the preparatory work for training a XGBoost model on the dataset, which is able to attain private and public scores of **0.86756** and **0.86104** respectively. The private and public scores are ranked top 100 and top 130 respectively 

More comprehensive README and Python scripts can be found at 

In [55]:
# Main tools we used:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Models:
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

# Python utilities:
import time
import os

In [35]:
# Import Data
print("Importing Raw Data...")
df_train = pd.read_csv("cs-training.csv")
df_test = pd.read_csv("cs-test.csv")
df_train.shape

Importing Raw Data...


(150000, 12)

## Preprocessing Suggestions <a class="anchor" id="4"></a>

- Remove samples with values of DebtRatio above its 97.5 percentile
- Set 0 <= RevolvingUtilizationOfUnsecuredLines < 13
- Set NumberOfTimes90DaysLate <= 17
- Impute MonthlyIncome with its median or mode, or with a normally distributed variable with MonthlyIncome's mean and standard deviation, or regression imputation.
- Impute NumberOfDependents with its mode

In [15]:
df_train.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [50]:
# Preprocess Data

## drop id column which doesn't provide any valuable info
df_train= df_train.drop("Unnamed: 0", axis=1)


In [51]:
df_train.shape

(150000, 11)

In [52]:
# Remove samples with values of DebtRatio above its 97.5 percentile
df_train_processed = df_train.loc[df_train["DebtRatio"] <= df_train["DebtRatio"].quantile(0.975)]
df_train_processed.shape

(146250, 11)

In [ ]:
df = df.loc[(df["RevolvingUtilizationOfUnsecuredLines"] >= 0) & (df["RevolvingUtilizationOfUnsecuredLines"] < 13)]
df = df.loc[df["NumberOfTimes90DaysLate"] <= 17]

**NumberOfDependents**

greater than 50% of the non-missing values have 0 dependents, and if someone leaves the field blank it is likely to be due to not having any dependents, we'll replace these values with 0.

In [46]:
# impute dependents with mode
dependents_mode = df_train["NumberOfDependents"].mode()[0] # impute with mode
df_train_processed["NumberOfDependents"] = df_train_processed["NumberOfDependents"].fillna(dependents_mode)
# df_train["NumberOfDependents"] = df_train["NumberOfDependents"].fillna(dependents_mode)
missing_values = df_train_processed["NumberOfDependents"].isna().sum()
print("Number of missing values for NumberOfDependents", missing_values)

Number of missing values for NumberOfDependents 0


**Handle missing data for monthly income**

When handling missing values for the "MonthlyIncome" column in a predictive modeling task,there are several options:

1. **Dropping rows**: If the number of missing values is relatively small compared to the total dataset, you can choose to drop the rows with missing values using the dropna() function. However, this approach may lead to loss of valuable data.

2. **Mean/Median/Mode** imputation: You can fill in the missing values with the mean, median, or mode of the "MonthlyIncome" column. For example, you can calculate the mean income and replace the missing values with that value using the fillna() function: df["MonthlyIncome"].fillna(df["MonthlyIncome"].mean(), inplace=True). Similarly, you can use the median or mode instead of the mean.

3. **Regression imputation**: If you have other variables in your dataset that are strongly correlated with the "MonthlyIncome" column, you can use regression techniques to predict the missing values based on those variables. You can train a regression model using the available data and then use the model to predict the missing values.

4. **Multiple imputation**: Multiple imputation is a technique where missing values are filled in with multiple plausible values, generating multiple complete datasets. Each dataset is then analyzed separately, and the results are combined to obtain the final estimates. This approach accounts for the uncertainty associated with imputing missing values.

In [53]:
missing_values = df_train_processed["MonthlyIncome"].isna().sum()
print("Number of missing values for MonthlyIncome", missing_values)

Number of missing values for MonthlyIncome 26166


First let's take a look at using regression to fill in the missing values for MonthlyIncome:

In [58]:
not_missing = not_missing = df_train_processed.dropna()
target = 'MonthlyIncome'
predictors = [c for c in list(not_missing) if c not in [target,'SeriousDlqin2yrs']]
X_data = not_missing[predictors]
y_data = not_missing[target]
regr = LinearRegression().fit(X_data, y_data)

In [59]:
regr.score(X_data, y_data)

0.023282823604717118

The R2 value obtained is extremely low, indicating poor model performance. Based on this observation, it appears that the most suitable approach would be to replace the missing values in the "MonthlyIncome" column with the median value instead of the mean. This decision is influenced by the presence of extreme outliers in the "MonthlyIncome" variable, where some individuals are earning exceptionally high incomes in the millions of dollars per month.

In [60]:

    #################################### Uncomment below if want median imputation for MonthlyIncome #################################### 
    income_median = df_train_processed["MonthlyIncome"].median()
    df_train_processed["MonthlyIncome"] = df_train_processed["MonthlyIncome"].fillna(income_median)
      
    ################################################################################################################################################ 


    #################################### Uncomment below if want random normal imputation for Monthly Income #################################### 
    # mean = df["MonthlyIncome"].mean()
    # std = df["MonthlyIncome"].std()
    # np.random.seed(0)
    # df.loc[df["MonthlyIncome"].isnull()]["MonthlyIncome"] = np.random.normal(loc=mean, scale=std, size=len(df.loc[df["MonthlyIncome"].isnull()]))
    # impute_dic = {
    #     "MonthlyIncome": {
    #         "mean": mean,
    #         "std": std
    #     },
    #     "NumberOfDependents": {
    #         "mode": dependents_mode
    #     }
    # }
    ################################################################################################################################################ 



<ipython-input-60-ec47db64df66>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_processed["MonthlyIncome"] = df_train_processed["MonthlyIncome"].fillna(income_median)


In [ ]:
df.to_csv(SAVE["preprocessed_data"], index=False)